In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy import signal
from time import sleep
import scipy.constants as spy
import sys
import math as m 

#setup some constants and parameters
M=2**21
tsim=7e-12
eps0 = 8.854187e-12
hbar = 1.05457182e-34
c = 2.99792458e8
kb = 1.380649e-23
hdk=7.63823258e-12
deltap = np.sqrt(1/3/eps0/hbar/c)*kb
T=50000

#define square root of Dn function
def Dnsqt(wla,T):
    return np.sqrt(6*hdk**2*wla/(np.exp(hdk*wla/T)-1)/T**2/np.pi)

#generate an array of the square root of Dn function
Dsqt = []
for n in range(1,M):
    Dsqt.append(Dnsqt(2*np.pi*n/tsim,T))

#define how many times to run to average the results
Ncomp=50000

#define the starting sum of the result
Efsum = np.zeros(2*M)

for i in range(Ncomp):
    
    #generate random numbers
    M0 = np.random.normal(0,1)
    Mlp = np.random.normal(0,1,M-1)
    Nlp = np.random.normal(0,1,M-1)
    MM = np.random.normal(0,1)
    
    #generate the array of the fourier transform of E field, we generate l=0 and l=-M separately. 
    #for l=1 to M-1 and l=-1 to -(M-1) are conjutated to each other. 
    #we contruct the whole array by gluing them together by the order from l=0 -> l=M-1 -> l=-M -> l=-(m-1) -> l=-1
    Ef0 = np.array([M0*np.sqrt(6*hdk/np.pi/T)])
    Eflp = np.multiply((Mlp + 1j*Nlp), Dsqt)
    Eflm = np.flip(np.conjugate(Eflp))
    EfM = np.array([MM*Dnsqt(2*np.pi*M/tsim,T)])
    Ef = np.hstack((Ef0,Eflp,EfM,Eflm))
    
    #we normalize Ef and compute its inverse fast fourier transform
    Efn = deltap*T/np.sqrt(tsim)*Ef
    En = np.fft.ifft(Efn)
    
    #we compute self correlation of En
    Eautoc = signal.correlate(En,En)
    Eautocf = np.fft.fft(Eautoc[len(En)-1:])
    
    #extract the real value and add the value of this loop to the total sum
    Eautocf = np.real(Eautocf)
    Efsum += Eautocf
    
    #generate some feedback to check the progress of the loop
    x = i/Ncomp*100
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%%" % ('😁'*int(x), int(x)))
    sys.stdout.flush()
    sleep(0.0001)

#average the final result
Efavg = Efsum/Ncomp

#save data to some file or reload file to write more data on it
np.savez("Efavg_v2.npz", Efavg_30000 = Efavg)
data = np.load("Efavg_v2.npz")
data = dict(data)
data["Efavg_50000"] = Efavg
np.savez("Efavg_v2.npz",**data)

[😁😁😁😁😁😁😁😁😁😁😁         ] 11%

In [ ]:
plt.plot(Efavg)
plt.xlim(0,50000)

In [ ]:
# Zeta = 
# E=<Es(t1)Es(t2)>
#t = t2-t1
T1= 30000
T2= 50000
#Delta
hb = (spy.h/(2*np.pi))
Epsilon0 = 8.85*10**(-12)

t_list = np.linspace(-1,1,10000)
t = t_list*10**(-15)            
a1 = spy.k*T1/hb
a2 = spy.k*T2/hb
def ES(Delta,t,a,T):
    def sigma1(k):
        return((k+1-1j*(t)*(spy.k*T/hb))**(-2))
    sum1 = 0
    for i in range(0,10000):
        sum1 += sigma1(i) 
        
    def sigma2(k):
        return((k+1+1j*(t)*(spy.k*T/hb))**(-2))
    sum2 = 0
    for i in range(0,10000):
        sum2 += sigma2(i) 
    return(3*((T*spy.k)*m.sqrt(2/Epsilon0/(6*hb*spy.c)))**(2)/np.pi**(2)*(sum1+sum2))
Delta1 = (T1*spy.k)*m.sqrt(1/Epsilon0/(6*hb*spy.c))
Delta2 = (T2*spy.k)*m.sqrt(1/Epsilon0/(6*hb*spy.c))
ES_30000 = ES(Delta1,t,a1,T1)
ES_50000 = ES(Delta2,t,a2,T2)            
plt_30000 = plt.plot(t*10**(15),ES_30000,label='30000K')
plt_50000 = plt.plot(t*10**(15),ES_50000,label='50000K')
plt.xlim(-1,1)
plt.ylim(0,0.6)
plt.xlabel ('Time (fs)')
plt.ylabel('Correlation function')
plt.legend()
plt.show()


In [ ]:
hb = (spy.h/(2*np.pi))
f_list=np.linspace(0, 25*10**(15),10000)
#f=f_list*10**(15)
def D_formul(f,T,):
    return((hb/(np.pi*spy.c))*((2*np.pi*f)/((np.e**(hb*(2*np.pi*f)/spy.k/T))-1)))
D_30000 = D_formul(f_list,T1)
D_50000 = D_formul(f_list,T2) 
f = f_list/10**(15)
plt_30000 = plt.plot(f*2*np.pi,D_30000,label='30000k')
plt_50000 = plt.plot(f*2*np.pi,D_50000,label='50000k')
plt.xlim(0,25)
plt.xlabel('omega?frequency(Hz)')
plt.ylabel('Energy density (10^-28 J/m)')
plt.legend()
plt.shoe()